In [5]:
from pathlib import Path
import requests
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
import time
import pandas as pd
import urllib
import re
import os
import re

In [6]:
def save_pdfs(pdfs, year):
    pdfs.reverse()
    for i, url in enumerate(pdfs):
        output_file = "DoRzeczy"+str(year)
        Path(output_file).mkdir(parents=True, exist_ok=True)
        filename = Path(f'{output_file}/dorzeczy_{i+1}_{year}.pdf')
        response = requests.get(url)
        filename.write_bytes(response.content)

In [7]:
chrome_options = Options()
chrome_options.add_argument("user-data-dir=selenium")
driver = webdriver.Chrome(options=chrome_options)

def get_url_mobi(link):
    driver.get(link)
    time.sleep(0.5)
    button = driver.find_element('link text',"pdf")
    button.click()
    time.sleep(0.5)



links =[]
begin = 411
end = 508
for i in range(end - begin):
        links.append(f"https://tygodnik.dorzeczy.pl/archiwum/{begin + i}/dorzeczy-{i+1}-{2022}.html")

print(links)
for link in links: pass #get_url_mobi(link)


['https://tygodnik.dorzeczy.pl/archiwum/411/dorzeczy-1-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/412/dorzeczy-2-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/413/dorzeczy-3-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/414/dorzeczy-4-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/415/dorzeczy-5-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/416/dorzeczy-6-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/417/dorzeczy-7-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/418/dorzeczy-8-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/419/dorzeczy-9-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/420/dorzeczy-10-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/421/dorzeczy-11-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/422/dorzeczy-12-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/423/dorzeczy-13-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/424/dorzeczy-14-2022.html', 'https://tygodnik.dorzeczy.pl/archiwum/425/dorzeczy-15-2022.html', 'ht

In [ ]:
def get_url(link):
    driver.get(link)
    button = driver.find_element('link text',"MOBI")
    button.click()
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[1])
    pdf_link = driver.current_url
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    print(pdf_link)
    return pdf_link

current_num = 536 - 29
nums = [51, 51, 52, 51, 51, 51, 51, 52, 51]
current_year = 2022

for num in nums:
    links =[]
    for i in range(num):
        links.append(f"https://tygodnik.dorzeczy.pl/archiwum/{current_num}/dorzeczy-{num - i}-{current_year}.html")
        current_num -= 1
    pdfs = [get_url(link) for link in links]
    save_pdfs(pdfs, current_year)
    current_year -= 1

In [ ]:


folder_path = 'DoRzeczy2014'  # Replace this with the path to your folder

# Define a regular expression pattern to match the files
pattern = r'^(dorzeczy)_(\d+)_(\d+\.pdf)$'

def rename_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            match = re.match(pattern, filename)
            if match:
                prefix, index, year = match.groups()
                new_index = int(index) - 2
                new_filename = f"{prefix}_{new_index}_{year}"
                new_file_path = os.path.join(folder_path, new_filename)
                os.rename(file_path, new_file_path)
                print(f"Renamed {filename} to {new_filename}")
            else:
                print(f"Skipped {filename} (doesn't match the pattern)")


#rename_files_in_folder(folder_path)

# DoRzeczy Dates

In [3]:
dates = set()

links = [f'https://eprasa.pl/news/do-rzeczy?page={i}' for i in range(2,8)]
links.append("https://eprasa.pl/news/do-rzeczy")

for link in links:
        with urllib.request.urlopen(link) as url:
            print(link)
            s = url.read().decode('utf-8')
            # I'm guessing this would output the html source code ?
            dates.update(set(re.findall("(?<=Do Rzeczy )(\d+) \(([\d\.]+)\)", s)))

dates.update(set([('17', '08.05.2022'),('17', '07.05.2023')]))

https://eprasa.pl/news/do-rzeczy?page=2
https://eprasa.pl/news/do-rzeczy?page=3
https://eprasa.pl/news/do-rzeczy?page=4
https://eprasa.pl/news/do-rzeczy?page=5
https://eprasa.pl/news/do-rzeczy?page=6
https://eprasa.pl/news/do-rzeczy?page=7
https://eprasa.pl/news/do-rzeczy


In [ ]:
df = pd.DataFrame(dates, columns=['magazine_nr', 'date'])
df['magazine_nr'] = df['magazine_nr'].astype(int)
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df.loc[(df.date.dt.year == 2023) & (df.magazine_nr > 17) & (df.magazine_nr < 21), 'magazine_nr'] -=1
df.loc[(df.date.dt.year == 2022) & (df.magazine_nr > 17), 'magazine_nr'] -=1

df.loc[df['date'].dt.year == 2022, 'magazine_nr'] += 456
df.loc[72, 'magazine_nr'] += 51
df.loc[df['date'].dt.year == 2021, 'magazine_nr'] += 405
df.loc[df['date'].dt.year == 2023, 'magazine_nr'] += 507
#df.to_csv('Extract/dorzeczy_dates.csv', index =False)

In [ ]:
df.to_csv("Extract/dorzeczy_dates.csv",index=False)

In [ ]:
df[df['date'].dt.year ==2021].sort_values('magazine_nr')

In [ ]:
df[df['magazine_nr'] == 473]['date']

In [ ]:
import datetime

double_date = datetime.date(2022,5,8)
df[df['date'].dt > double_date]